In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
Measurement = Base.classes.measurement

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [11]:
for row in session.query(Measurement, Measurement.date).limit(5).all():
    print(row)

(<sqlalchemy.ext.automap.measurement object at 0x0000012143930FC8>, '2010-01-01')
(<sqlalchemy.ext.automap.measurement object at 0x0000012143930DC8>, '2010-01-02')
(<sqlalchemy.ext.automap.measurement object at 0x00000121439154C8>, '2010-01-03')
(<sqlalchemy.ext.automap.measurement object at 0x00000121439006C8>, '2010-01-04')
(<sqlalchemy.ext.automap.measurement object at 0x0000012143900C08>, '2010-01-06')


# Exploratory Climate Analysis

In [12]:
class Perc(Base):
    __tablename__ = 'Percip'
    id = Column(Integer, primary_key=True) 
    station = Column(String) 
    name = Column(String) 
    latitude = Column(Integer) 
    longitude = Column(Integer) 
    elevation = Column(Integer)
    #id INTEGER NOT NULL, 
    station = Column(String) 
    date = Column(String)  
    prcp = Column(Integer) 
    tobs  = Column(Integer)
    

In [13]:
inspector = inspect(engine)

In [14]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [15]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

sel = [ 
       (Measurement.date), 
       (Measurement.prcp)]
year_averages = session.query(*sel).\
    filter(func.strftime("%y", Measurement.date) == "2019").\
    group_by(Measurement.date).\
    order_by(Measurement.prcp).all()

year_averages


[]

In [16]:
df = pd.DataFrame(year_averages, columns=['date', 'prcp'])
df.set_index('date', inplace=True)
df.plot.bar()
plt.tight_layout()
plt.show()


TypeError: no numeric data to plot

In [ ]:
# Perform a query to retrieve the data and precipitation scores

results = session.query(Measurement.date, Measurement.prcp).\
    order_by(Measurement.date.desc()).all()

results 


In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column


In [ ]:
# Sort the dataframe by date


In [28]:
# Use Pandas Plotting with Matplotlib to plot the data


In [15]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [16]:
# Design a query to show how many stations are available in this dataset?
stations = session.query(Measurement).group_by(Measurement.station).count()
print(stations)

9


In [17]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

for row in session.query(Station, Station.name).limit(5).all():
    print(row)


(<sqlalchemy.ext.automap.station object at 0x0000026224FED148>, 'WAIKIKI 717.2, HI US')
(<sqlalchemy.ext.automap.station object at 0x0000026224FEDA48>, 'KANEOHE 838.1, HI US')
(<sqlalchemy.ext.automap.station object at 0x0000026224FEDC48>, 'KUALOA RANCH HEADQUARTERS 886.9, HI US')
(<sqlalchemy.ext.automap.station object at 0x0000026224FE8508>, 'PEARL CITY, HI US')
(<sqlalchemy.ext.automap.station object at 0x0000026224FE8748>, 'UPPER WAHIAWA 874.3, HI US')


In [18]:
engine.execute('SELECT * FROM measurement LIMIT 10').fetchall()



[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (10, 'USC00519397', '2010-01-11', 0.01, 64.0)]

In [22]:
#Select only the date and prcp values.



df = pd.DataFrame(results[:10], columns=['station', 'date', 'prcp'])
df.set_index('station', inplace=True, )
df.head(10)

,date,prcp
station,,
USC00519397,0.01,87.0
USC00519397,0.08,87.0
USC00517948,NaN,87.0
USC00511918,0.00,87.0
USC00511918,0.00,86.0
USC00511918,0.00,86.0
USC00511918,0.00,86.0
USC00511918,NaN,86.0
USC00519397,0.00,85.0


In [23]:
station = df.iloc[:,0]

In [24]:
station.head()

station
USC00519397    0.01
USC00519397    0.08
USC00517948     NaN
USC00511918    0.00
USC00511918    0.00
Name: date, dtype: float64

In [25]:
station.value_counts('station')

0.00    0.750
0.01    0.125
0.08    0.125
Name: date, dtype: float64

In [73]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?

for row in session.query(Measurement, Measurement.tobs).limit(5).all():
    print(row)
    
    



(<sqlalchemy.ext.automap.measurement object at 0x0000022BAE8E9208>, 65.0)
(<sqlalchemy.ext.automap.measurement object at 0x0000022BAE8E9348>, 63.0)
(<sqlalchemy.ext.automap.measurement object at 0x0000022BAE8E93C8>, 74.0)
(<sqlalchemy.ext.automap.measurement object at 0x0000022BAE8E9448>, 76.0)
(<sqlalchemy.ext.automap.measurement object at 0x0000022BAE8DFCC8>, 73.0)


In [58]:
station.count()

8

In [66]:
emoji_id = [result[1] for result in results[:10]]
scores = [int(result[2]) for result in results[:10]]

In [ ]:
sel = [ 
       func.avg(Measurement.date), 
       func.avg(Measurement.prcp)]
year_averages = session.query(*sel).\
    filter(func.strftime("%y", Measurement.date) == "2019").\
    group_by(Measurement.date).\
    order_by(Measurement.prcp).all()
year_averages

In [72]:
x_axis = np.arange(len(station))
tick_locations = [value for value in x_axis]

In [74]:
plt.figure(figsize=(20,3))
plt.bar(x_axis, station["prcp"], color='r', alpha=0.5, align="center")
plt.xticks(tick_locations, station["date"], rotation="vertical")

KeyError: 'prcp'

<Figure size 1440x216 with 0 Axes>

In [75]:
for row in session.query(Measurement, Measurement.tobs).min():
    print(row)

AttributeError: 'Query' object has no attribute 'min'

In [1]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


![precipitation](Images/station-histogram.png)